In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## Dataset loading (pre-split into 70/30 - Train/Test)

In [2]:
test_data = pd.read_csv('test_data.csv')

In [3]:
test_data.shape

(17133, 31)

In [4]:
test_data['id'].nunique()

1032

In [5]:
train_data = pd.read_csv('train_data.csv')

In [6]:
train_data['id'].nunique()

2419

In [7]:
train_data.shape

(38821, 31)

## Calculation of the table
- "baseline values" - parameter values at admission to the ICU
- p-value - t-test (Welch)

In [8]:


LAB_COLUMNS = [
    'Aging', 'Gender',
    'ALT','APTT','AST','Amilasa','CRP','Conj_bilirubin','Creatinin',
    'D_dimer','Ferritin','Fibrinogen','Glucose','Hemoglobin','LDG',
    'Lymphocytes','Monocytes','Neutrophils','Platelets','Potassium',
    'Procalcitonin','Sodium','Total_protein','Urea','WBC'
]

from scipy import stats
import numpy as np

# -------- p-value for comparing two proportions using a t-test (Welch) --------
def ttest_proportions_p(x1, n1, x2, n2):
    if min(n1, n2) == 0:
        return np.nan
    p1 = x1 / n1
    p2 = x2 / n2
    # standard error for proportion difference
    var1 = p1 * (1 - p1) / n1
    var2 = p2 * (1 - p2) / n2
    se2 = var1 + var2
    if se2 <= 0:
        # equal proportions and zero variance
        return 1.0 if abs(p1 - p2) < 1e-12 else np.nan
    t_stat = (p1 - p2) / np.sqrt(se2)
    # Satterthwaite degrees of freedom
    with np.errstate(divide='ignore', invalid='ignore'):
        df_num = se2 ** 2
        df_den = (var1 ** 2) / (n1 - 1 if n1 > 1 else np.inf) + (var2 ** 2) / (n2 - 1 if n2 > 1 else np.inf)
        df = df_num / df_den if df_den > 0 else np.inf
    p = 2 * (1 - stats.t.cdf(abs(t_stat), df))
    return float(p)


def mad(series: pd.Series) -> float:
    s = pd.to_numeric(series, errors='coerce').dropna()
    if s.empty: return np.nan
    med = np.median(s)
    return float(np.median(np.abs(s - med)))

def _base_days(df, id_col, day_col):
    days = pd.to_numeric(df[day_col], errors='coerce')
    return (df.loc[days.notna(), [id_col, day_col]]
              .groupby(id_col, as_index=False)[day_col].max()
              .rename(columns={day_col: 'd0'}))

def _baseline_series_with_fallback(df, col, id_col, day_col, base_days,
                                   cast_numeric=True, use_prev_day=True):
    tmp = df[[id_col, day_col, col]].merge(base_days, on=id_col, how='right')
    tmp[day_col] = pd.to_numeric(tmp[day_col], errors='coerce').round().astype('Int64')
    tmp['d0']     = pd.to_numeric(tmp['d0'], errors='coerce').round().astype('Int64')
    if cast_numeric:
        tmp[col] = pd.to_numeric(tmp[col], errors='coerce')

    if use_prev_day:
        m = (tmp[day_col] == tmp['d0']) | (tmp[day_col] == (tmp['d0'] - 1))
        tmp = tmp.loc[m].copy()
        tmp['prio'] = np.where(tmp[day_col] == tmp['d0'], 0, 1)
    else:
        tmp = tmp.loc[tmp[day_col] == tmp['d0']].copy()
        tmp['prio'] = 0

    tmp = tmp.sort_values([id_col, 'prio'])
    s = tmp.dropna(subset=[col]).groupby(id_col)[col].first()
    return s.reindex(base_days[id_col])

def _per_patient_interval_means(df, col, id_col, day_col):
    days = pd.to_numeric(df[day_col], errors='coerce')
    vals = pd.to_numeric(df[col], errors='coerce')
    dfv = df.loc[vals.notna() & days.notna(), [id_col, day_col]].drop_duplicates()
    out = []
    for _, g in dfv.groupby(id_col):
        d = np.sort(g[day_col].unique())
        if len(d) > 1:
            iv = np.diff(d)
            out.append(iv.mean())
    return np.array(out, dtype=float)

def _format_pct(n, total):
    if not total or total == 0: return "0 (0.0%)"
    return f"{n} ({n/total*100:.1f}%)"

def build_baseline_table_two_cohorts(train_df: pd.DataFrame,
                                     test_df: pd.DataFrame,
                                     lab_columns=LAB_COLUMNS,
                                     id_col='id',
                                     day_col='DaysBfOutcome') -> pd.DataFrame:
    # preprocessing of base days
    base_train = _base_days(train_df, id_col, day_col)
    base_test  = _base_days(test_df,  id_col, day_col)
    n_train = train_df[id_col].nunique()
    n_test  = test_df[id_col].nunique()

    rows = []

    for col in lab_columns:
        if col not in train_df.columns or col not in test_df.columns:
            continue

        # ====== GENDER ======
        if col == 'Gender':
            g_tr = _baseline_series_with_fallback(train_df, col, id_col, day_col, base_train,
                                                  cast_numeric=False, use_prev_day=True)
            g_te = _baseline_series_with_fallback(test_df,  col, id_col, day_col, base_test,
                                                  cast_numeric=False, use_prev_day=True)

            def norm_gender(s):
                s = s.astype('string').str.strip().str.upper()
                mp = {'FEMALE':'F','F':'F','ЖЕН':'F','Ж':'F','WOMAN':'F',
                      'MALE':'M','M':'M','МУЖ':'M','М':'M','MAN':'M'}
                s = s.replace(mp)
                s = s.replace({'': pd.NA, 'NA': pd.NA, 'N/A': pd.NA, 'NONE': pd.NA, 'NAN': pd.NA})
                return s

            g_tr = norm_gender(g_tr); g_te = norm_gender(g_te)
            n_tr = int(g_tr.notna().sum()); n_te = int(g_te.notna().sum())
            f_tr = int((g_tr=='F').sum()); m_tr = int((g_tr=='M').sum())
            f_te = int((g_te=='F').sum()); m_te = int((g_te=='M').sum())

            # p-value via a t-test for the difference of fractions (women)
            if (n_tr > 0) and (n_te > 0):
                p_gender = ttest_proportions_p(f_tr, f_tr + m_tr, f_te, f_te + m_te)
            else:
                p_gender = np.nan

            rows.append({
                'Test': col,

                'Patients with known baseline values (1)': n_tr,
                'Patients with known baseline values (2)': n_te,
                'Patients with known baseline values (p)': np.nan,

                'Patients tested at least once (1)': n_tr,
                'Patients tested at least once (2)': n_te,
                'Patients tested at least once (p)': np.nan,

                'Average testing period (days) (1)': np.nan,
                'Average testing period (days) (2)': np.nan,
                'Average testing period (days) (p)': np.nan,

                'Baseline values median (MAD) (1)': f"F: {_format_pct(f_tr,n_tr)}; M: {_format_pct(m_tr,n_tr)}",
                'Baseline values median (MAD) (2)': f"F: {_format_pct(f_te,n_te)}; M: {_format_pct(m_te,n_te)}",
                'Baseline values median (MAD) (p)': p_gender,
            })
            continue

        # ====== AGING ======
        if col == 'Aging':
            b_tr = _baseline_series_with_fallback(train_df, col, id_col, day_col, base_train,
                                                  cast_numeric=True, use_prev_day=True)
            b_te = _baseline_series_with_fallback(test_df,  col, id_col, day_col, base_test,
                                                  cast_numeric=True, use_prev_day=True)
            n_bl_tr = int(b_tr.notna().sum()); n_bl_te = int(b_te.notna().sum())

            # t-test (Welch) by baseline
            if n_bl_tr > 0 and n_bl_te > 0:
                p_med = stats.ttest_ind(b_tr.dropna(), b_te.dropna(), equal_var=False).pvalue
            else:
                p_med = np.nan

            rows.append({
                'Test': col,

                'Patients with known baseline values (1)': n_bl_tr,
                'Patients with known baseline values (2)': n_bl_te,
                'Patients with known baseline values (p)': ttest_proportions_p(n_bl_tr, n_train, n_bl_te, n_test),

                'Patients tested at least once (1)': n_bl_tr,
                'Patients tested at least once (2)': n_bl_te,
                'Patients tested at least once (p)': np.nan,

                'Average testing period (days) (1)': np.nan,
                'Average testing period (days) (2)': np.nan,
                'Average testing period (days) (p)': np.nan,

                'Baseline values median (MAD) (1)': f"{np.nanmedian(b_tr):.2f} ({mad(b_tr):.2f})" if n_bl_tr else np.nan,
                'Baseline values median (MAD) (2)': f"{np.nanmedian(b_te):.2f} ({mad(b_te):.2f})" if n_bl_te else np.nan,
                'Baseline values median (MAD) (p)': p_med,
            })
            continue

        # ====== The usual numerical tests ======
        # tested at least once
        tr_vals = pd.to_numeric(train_df[col], errors='coerce')
        te_vals = pd.to_numeric(test_df[col],  errors='coerce')
        n_tested_tr = int(train_df.loc[tr_vals.notna(), id_col].nunique())
        n_tested_te = int(test_df .loc[te_vals.notna(), id_col].nunique())

        # p for "tested at least once" — T-proportion test
        p_tested = ttest_proportions_p(n_tested_tr, n_train, n_tested_te, n_test)

        # baseline со fallback d0 / d0-1
        b_tr = _baseline_series_with_fallback(train_df, col, id_col, day_col, base_train,
                                              cast_numeric=True, use_prev_day=True)
        b_te = _baseline_series_with_fallback(test_df,  col, id_col, day_col, base_test,
                                              cast_numeric=True, use_prev_day=True)
        n_bl_tr = int(b_tr.notna().sum()); n_bl_te = int(b_te.notna().sum())

        # p for known baseline (fractions) — t-test of proportions
        p_known  = ttest_proportions_p(n_bl_tr, n_train, n_bl_te, n_test)

        # p for baseline values — t-test (Welch)
        if n_bl_tr > 0 and n_bl_te > 0:
            p_med = stats.ttest_ind(b_tr.dropna(), b_te.dropna(), equal_var=False).pvalue
        else:
            p_med = np.nan

        # "Average testing period": for the patient, the average interval → comparison by t-test
        iv_tr = _per_patient_interval_means(train_df, col, id_col, day_col)
        iv_te = _per_patient_interval_means(test_df,  col, id_col, day_col)
        avg_tr = float(np.mean(iv_tr)) if iv_tr.size else np.nan
        avg_te = float(np.mean(iv_te)) if iv_te.size else np.nan
        if iv_tr.size and iv_te.size:
            p_iv = stats.ttest_ind(iv_tr, iv_te, equal_var=False).pvalue
        else:
            p_iv = np.nan

        rows.append({
            'Test': col,

            'Patients with known baseline values (1)': n_bl_tr,
            'Patients with known baseline values (2)': n_bl_te,
            'Patients with known baseline values (p)': p_known,

            'Patients tested at least once (1)': n_tested_tr,
            'Patients tested at least once (2)': n_tested_te,
            'Patients tested at least once (p)': p_tested,

            'Average testing period (days) (1)': None if np.isnan(avg_tr) else round(avg_tr, 1),
            'Average testing period (days) (2)': None if np.isnan(avg_te) else round(avg_te, 1),
            'Average testing period (days) (p)': p_iv,

            'Baseline values median (MAD) (1)': (
                f"{np.nanmedian(b_tr):.2f} ({mad(b_tr):.2f})" if n_bl_tr else np.nan
            ),
            'Baseline values median (MAD) (2)': (
                f"{np.nanmedian(b_te):.2f} ({mad(b_te):.2f})" if n_bl_te else np.nan
            ),
            'Baseline values median (MAD) (p)': p_med,
        })

    out = pd.DataFrame(rows).set_index('Test')

    # round up p and add NaN
    def fmt_p(x):
        if pd.isna(x): return np.nan
        return "<0.001" if x < 1e-3 else round(float(x), 3)

    for col in out.columns:
        if col.endswith('(p)'):
            out[col] = out[col].apply(fmt_p)

    return out


In [9]:
table_2c = build_baseline_table_two_cohorts(train_data, test_data,
                                            lab_columns=LAB_COLUMNS,
                                            id_col='id', day_col='DaysBfOutcome')
display(table_2c)


,Patients with known baseline values (1),Patients with known baseline values (2),Patients with known baseline values (p),Patients tested at least once (1),Patients tested at least once (2),Patients tested at least once (p),Average testing period (days) (1),Average testing period (days) (2),Average testing period (days) (p),Baseline values median (MAD) (1),Baseline values median (MAD) (2),Baseline values median (MAD) (p)
Test,,,,,,,,,,,,
Aging,2419,1032,1.000,2419,1032,NaN,NaN,NaN,NaN,64.00 (10.00),64.50 (10.50),0.643
Gender,2419,1032,NaN,2419,1032,NaN,NaN,NaN,NaN,F: 912 (37.7%); M: 1506 (62.3%),F: 412 (39.9%); M: 620 (60.1%),0.225
ALT,2135,916,0.672,2415,1029,0.502,1.9,2.0,0.365,29.00 (13.00),28.00 (12.00),0.267
APTT,1938,852,0.088,2299,1001,0.005,1.9,2.0,0.244,26.23 (3.73),26.00 (4.00),0.803
AST,2136,916,0.698,2415,1029,0.502,1.9,2.0,0.390,38.00 (17.00),37.45 (16.45),0.077
Amilasa,1522,624,0.176,2138,934,0.059,2.7,2.8,0.575,54.00 (23.00),51.60 (22.60),0.306
CRP,155,66,0.989,430,192,0.565,6.1,8.6,0.124,48.00 (38.00),48.00 (36.00),0.572
Conj_bilirubin,2107,905,0.630,2414,1028,0.399,2.0,2.0,0.437,3.50 (1.50),3.00 (1.50),0.631
Creatinin,1930,846,0.130,2381,1019,0.469,2.1,2.1,0.438,92.00 (23.85),92.00 (23.00),0.517


## Making pretty table

In [10]:
import re
import pandas as pd

def add_percents_and_group_headers(table_2c: pd.DataFrame, n1: int, n2: int) -> pd.DataFrame:

    df = table_2c.copy()

    def fmt_n_pct(x, total):
        if pd.isna(x) or not total:
            return "NA"
        x = int(x)
        return f"{x} ({x/total*100:.1f}%)"

    blocks_to_format = [
        'Patients with known baseline values',
        'Patients tested at least once',
    ]
    for block in blocks_to_format:
        c1 = f"{block} (1)"
        c2 = f"{block} (2)"
        if c1 in df.columns: df[c1] = df[c1].apply(lambda v: fmt_n_pct(v, n1))
        if c2 in df.columns: df[c2] = df[c2].apply(lambda v: fmt_n_pct(v, n2))

    group_order = [
        'Patients with known baseline values',
        'Patients tested at least once',
        'Average testing period (days)',
        'Baseline values median (MAD)',
    ]
    sub_order = ['1','2','p']

    new_cols = []
    for c in df.columns:
        m = re.match(r'^(.*)\s\((1|2|p)\)$', c)
        if m:
            group, sub = m.group(1), m.group(2)
        else:
            group, sub = c, ''
        new_cols.append((group, sub))
    df.columns = pd.MultiIndex.from_tuples(new_cols, names=['', ''])

    def sort_key(t):
        g, s = t
        gi = group_order.index(g) if g in group_order else len(group_order)
        si = sub_order.index(s) if s in sub_order else len(sub_order)
        return (gi, si)

    df = df.reindex(columns=sorted(df.columns, key=sort_key))
    return df


n_train = train_data['id'].nunique()
n_test  = test_data['id'].nunique()

pretty = add_percents_and_group_headers(table_2c, n_train, n_test)
display(pretty)

pretty.to_excel('baseline_two_cohorts.xlsx', merge_cells=True)
with open('baseline_two_cohorts.html','w',encoding='utf-8') as f:
    f.write(pretty.style.format(na_rep="NA").to_html())


Patients with known baseline values                        \
                                                 1              2      p   
Test                                                                       
Aging                                2419 (100.0%)  1032 (100.0%)  1.000   
Gender                               2419 (100.0%)  1032 (100.0%)    NaN   
ALT                                   2135 (88.3%)    916 (88.8%)  0.672   
APTT                                  1938 (80.1%)    852 (82.6%)  0.088   
AST                                   2136 (88.3%)    916 (88.8%)  0.698   
Amilasa                               1522 (62.9%)    624 (60.5%)  0.176   
CRP                                     155 (6.4%)      66 (6.4%)  0.989   
Conj_bilirubin                        2107 (87.1%)    905 (87.7%)  0.630   
Creatinin                             1930 (79.8%)    846 (82.0%)  0.130   
D_dimer                                946 (39.1%)    395 (38.3%)  0.646   
Ferritin                                 11 (0.5%)       9 (0.9%)  0.193   
Fibrinogen                            2071 (85.6%)    895 (86.7%)  0.384   
Glucose                               2238 (92.5%)    960 (93.0%)  0.597   
Hemoglobin                            2362 (97.6%)   1009 (97.8%)  0.818   
LDG                                    484 (20.0%)    231 (22.4%)  0.121   
Lymphocytes                           2084 (86.2%)    908 (88.0%)  0.137   
Monocytes                             2081 (86.0%)    909 (88.1%)  0.095   
Neutrophils                           2085 (86.2%)    910 (88.2%)  0.105   
Platelets                             2325 (96.1%)    994 (96.3%)  0.773   
Potassium                              252 (10.4%)    105 (10.2%)  0.829   
Procalcitonin                          885 (36.6%)    360 (34.9%)  0.339   
Sodium                                 252 (10.4%)    105 (10.2%)  0.829   
Total_protein                         2133 (88.2%)    913 (88.5%)  0.806   
Urea                                   335 (13.8%)    145 (14.1%)  0.876   
WBC                                   2364 (97.7%)   1011 (98.0%)  0.655   

               Patients tested at least once                        \
                                           1              2      p   
Test                                                                 
Aging                          2419 (100.0%)  1032 (100.0%)    NaN   
Gender                         2419 (100.0%)  1032 (100.0%)    NaN   
ALT                             2415 (99.8%)   1029 (99.7%)  0.502   
APTT                            2299 (95.0%)   1001 (97.0%)  0.005   
AST                             2415 (99.8%)   1029 (99.7%)  0.502   
Amilasa                         2138 (88.4%)    934 (90.5%)  0.059   
CRP                              430 (17.8%)    192 (18.6%)  0.565   
Conj_bilirubin                  2414 (99.8%)   1028 (99.6%)  0.399   
Creatinin                       2381 (98.4%)   1019 (98.7%)  0.469   
D_dimer                         1258 (52.0%)    535 (51.8%)  0.930   
Ferritin                           62 (2.6%)      30 (2.9%)  0.575   
Fibrinogen                      2414 (99.8%)   1028 (99.6%)  0.399   
Glucose                         2417 (99.9%)   1031 (99.9%)  0.900   
Hemoglobin                     2419 (100.0%)  1032 (100.0%)  1.000   
LDG                              793 (32.8%)    349 (33.8%)  0.555   
Lymphocytes                     2416 (99.9%)   1030 (99.8%)  0.652   
Monocytes                       2416 (99.9%)   1030 (99.8%)  0.652   
Neutrophils                     2416 (99.9%)   1030 (99.8%)  0.652   
Platelets                       2417 (99.9%)   1031 (99.9%)  0.900   
Potassium                        520 (21.5%)    215 (20.8%)  0.662   
Procalcitonin                   1346 (55.6%)    571 (55.3%)  0.865   
Sodium                           520 (21.5%)    215 (20.8%)  0.662   
Total_protein                   2415 (99.8%)   1029 (99.7%)  0.502   
Urea                             637 (26.3%)    258 (25.0%)  0.410

## Export pretty table for R

In [11]:
import pandas as pd
import json

def export_multiindex_for_r_with_test(pretty: pd.DataFrame,
                                      csv_path: str = "pretty_table.csv",
                                      meta_path: str = "pretty_table_meta.json"):
    
    df = pretty.reset_index()
    if df.columns[0] != 'Test':
        df = df.rename(columns={df.columns[0]: 'Test'})

    meta = []
    meta.append(["", "Test"])

    for col in pretty.columns:
        if isinstance(col, tuple):

            group = str(col[0])
            sub   = str(col[1]) if len(col) > 1 else ""
            meta.append([group, sub])
        else:
            meta.append(["", str(col)])

    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    flat = df.copy()
    flat.columns = [
        " | ".join(map(str, c)) if isinstance(c, tuple) else str(c)
        for c in df.columns
    ]
    flat.to_csv(csv_path, index=False)

# пример
export_multiindex_for_r_with_test(pretty, "pretty_table.csv", "pretty_table_meta.json")
